In [1]:
import os
import numpy as np
import subprocess
import pymesh
import tempfile, shutil
#import Bio.PDB
from Bio.PDB import PDBParser, PDBIO, Select 
from Bio.PDB import NeighborSearch, Selection
from rdkit import Chem
from scipy.spatial import distance, KDTree
from IPython.utils import io
from joblib import Parallel, delayed
#from deepdock.utils.mol2graph import *

#os.environ["BABEL_LIBDIR"] = "/home/shenchao/.conda/envs/deepdock/lib/openbabel/3.1.0"
#from openbabel import pybel

import sys
#sys.path.append(os.path.dirname(__file__))
from compute_normal import compute_normal
from computeAPBS import computeAPBS
from computeCharges import computeCharges, assignChargesToNewMesh
from computeHydrophobicity import computeHydrophobicity
from computeMSMS import computeMSMS
from fixmesh import fix_mesh
from save_ply import save_ply

In [4]:
prot_path = '/home/haotian/Molecule_Generation/Surface-BP/dataset/masif/1z6e_protein.pdb'
lig_path = '/home/haotian/Molecule_Generation/Surface-BP/dataset/masif/1z6e_ligand.mol2'
dist_threshold=10.0
mesh_res=1.0
epsilon=1.0e-6

In [37]:

workdir = '/home/haotian/Molecule_Generation/Surface-BP/dataset/masif/workdir'
protname = os.path.basename(prot_path).replace(".pdb","")
mol = Chem.MolFromMol2File(lig_path)

In [38]:
atomCoords = mol.GetConformers()[0].GetPositions()
# Read protein and select aminino acids in the binding pocket
parser = PDBParser(QUIET=True) # QUIET=True avoids comments on errors in the pdb.

structures = parser.get_structure('target', prot_path)
structure = structures[0] # 'structures' may contain several proteins in this case only one.

atoms  = Selection.unfold_entities(structure, 'A')
ns = NeighborSearch(atoms)

close_residues= []
for a in atomCoords:  
    close_residues.extend(ns.search(a, dist_threshold+5, level='R'))
close_residues = Selection.uniqueify(close_residues)

class SelectNeighbors(Select):
    def accept_residue(self, residue):
        if residue in close_residues:
            if all(a in [i.get_name() for i in residue.get_unpacked_list()] for a in ['N', 'CA', 'C', 'O']) or residue.resname=='HOH':
                return True
            else:
                return False
        else:
            return False
    
pdbio = PDBIO()
pdbio.set_structure(structure)
pdbio.save("%s/%s_pocket_%s.pdb"%(workdir, protname, dist_threshold+5), SelectNeighbors())

In [39]:
structures = parser.get_structure('target', "%s/%s_pocket_%s.pdb"%(workdir, protname, dist_threshold+5))
structure = structures[0] # 'structures' may contain several proteins in this case only one.
atoms = Selection.unfold_entities(structure, 'A')

In [41]:
msms_bin="/usr/local/bin/msms"
apbs_bin="/home/haotian/Molecule_Generation/APBS-3.0.0.Linux/bin/apbs"
pdb2pqr_bin="/home/haotian/Molecule_Generation/pdb2pqr-linux-bin64-2.1.1/pdb2pqr"
multivalue_bin="/home/haotian/Molecule_Generation/APBS-3.0.0.Linux/share/apbs/tools/bin/multivalue"

In [42]:
dist = [distance.euclidean(atomCoords.mean(axis=0), a.get_coord()) for a in atoms]
atom_idx = np.argmin(dist)
vertices1, faces1, normals1, names1, areas1 = computeMSMS("%s/%s_pocket_%s.pdb"%(workdir, protname, dist_threshold+5),  
                                                            protonate=True, 
                                                            one_cavity=atom_idx, 
                                                            msms_bin=msms_bin,
                                                            workdir=workdir)
                                                                
# Find the distance between every vertex in binding site surface and each atom in the ligand.
kdt = KDTree(atomCoords)
d, r = kdt.query(vertices1)
assert(len(d) == len(vertices1))
iface_v = np.where(d <= dist_threshold)[0]
faces_to_keep = [idx for idx, face in enumerate(faces1) if all(v in iface_v  for v in face)] 

# Compute "charged" vertices

vertex_hbond = computeCharges(prot_path.replace(".pdb",""), vertices1, names1)    

# For each surface residue, assign the hydrophobicity of its amino acid. 

vertex_hphobicity = computeHydrophobicity(names1) 

vertices2 = vertices1
faces2 = faces1
# Fix the mesh.
mesh = pymesh.form_mesh(vertices2, faces2)
mesh = pymesh.submesh(mesh, faces_to_keep, 0)
with io.capture_output() as captured:
    regular_mesh = fix_mesh(mesh, mesh_res)


In [43]:
regular_mesh, info = pymesh.remove_degenerated_triangles(regular_mesh)

# Compute the normals
vertex_normal = compute_normal(regular_mesh.vertices, regular_mesh.faces, eps=epsilon)

In [44]:
vertex_hbond = assignChargesToNewMesh(regular_mesh.vertices, vertices1, vertex_hbond, True)

In [45]:
vertex_hphobicity = assignChargesToNewMesh(regular_mesh.vertices, vertices1, vertex_hphobicity, True)

In [46]:
vertex_charges = computeAPBS(regular_mesh.vertices, '/home/haotian/Molecule_Generation/Surface-BP/dataset/masif/workdir/1z6e_protein_pocket_15.0.pdb', 
									apbs_bin, pdb2pqr_bin, multivalue_bin, workdir)

FileNotFoundError: [Errno 2] No such file or directory: '/home/haotian/Molecule_Generation/Surface-BP/dataset/masif/workdir/temp1_out.csv'

In [27]:
chargefile = open("%s/temp1_out.csv"%workdir)

In [33]:
charges = np.array([0.0] * len(regular_mesh.vertices))
for ix, line in enumerate(chargefile.readlines()):
    charges[ix] = float(line.split(",")[3])

In [ ]:
import os
import numpy as np
import subprocess
import pymesh
import tempfile, shutil
#import Bio.PDB
from Bio.PDB import PDBParser, PDBIO, Select 
from Bio.PDB import NeighborSearch, Selection
from rdkit import Chem
from scipy.spatial import distance, KDTree
from IPython.utils import io
from joblib import Parallel, delayed
#from deepdock.utils.mol2graph import *

#os.environ["BABEL_LIBDIR"] = "/home/shenchao/.conda/envs/deepdock/lib/openbabel/3.1.0"
#from openbabel import pybel

import sys
#sys.path.append(os.path.dirname(__file__))
from compute_normal import compute_normal
from computeAPBS import computeAPBS
from computeCharges import computeCharges, assignChargesToNewMesh
from computeHydrophobicity import computeHydrophobicity
from computeMSMS import computeMSMS
from fixmesh import fix_mesh
from save_ply import save_ply

In [ ]:
prot_path = '/home/haotian/molecules_confs/SDEGen-Plus/sdegen/feats/masif/1z6e_protein.pdb'
lig_path = '/home/haotian/molecules_confs/SDEGen-Plus/sdegen/feats/masif/1z6e_ligand.mol2'
outdir = '.'
dist_threshold=10.0
use_hbond=True
use_hphob=True
use_apbs=True
compute_iface=True
mesh_res=1.0
epsilon=1.0e-6
feature_interpolation=True

In [ ]:
workdir = tempfile.mkdtemp()
protname = os.path.basename(prot_path).replace(".pdb","")
mol = Chem.MolFromMol2File(lig_path)
atomCoords = mol.GetConformers()[0].GetPositions()
parser = PDBParser(QUIET=True) # QUIET=True avoids comments on errors in the pdb.

structures = parser.get_structure('target', prot_path)
structure = structures[0] # 'structures' may contain several proteins in this case only one.

atoms  = Selection.unfold_entities(structure, 'A')
ns = NeighborSearch(atoms)

close_residues= []
for a in atomCoords:  
    close_residues.extend(ns.search(a, dist_threshold+5, level='R'))
close_residues = Selection.uniqueify(close_residues)

class SelectNeighbors(Select):
    def accept_residue(self, residue):
        if residue in close_residues:
            if all(a in [i.get_name() for i in residue.get_unpacked_list()] for a in ['N', 'CA', 'C', 'O']) or residue.resname=='HOH':
                return True
            else:
                return False
        else:
            return False
    
pdbio = PDBIO()
pdbio.set_structure(structure)
pdbio.save("%s/%s_pocket_%s.pdb"%(workdir, protname, dist_threshold+5), SelectNeighbors())

# Identify closes atom to the ligand
structures = parser.get_structure('target', "%s/%s_pocket_%s.pdb"%(workdir, protname, dist_threshold+5))
structure = structures[0] # 'structures' may contain several proteins in this case only one.
atoms = Selection.unfold_entities(structure, 'A')

In [ ]:

dist = [distance.euclidean(atomCoords.mean(axis=0), a.get_coord()) for a in atoms]
atom_idx = np.argmin(dist)
vertices1, faces1, normals1, names1, areas1 = computeMSMS("%s/%s_pocket_%s.pdb"%(workdir, protname, dist_threshold+5),  
                                                            protonate=True, 
                                                            one_cavity=atom_idx, 
                                                            msms_bin=msms_bin,
                                                            workdir=workdir)

In [ ]:
kdt = KDTree(atomCoords)
d, r = kdt.query(vertices1)
assert(len(d) == len(vertices1))
iface_v = np.where(d <= dist_threshold)[0]
faces_to_keep = [idx for idx, face in enumerate(faces1) if all(v in iface_v  for v in face)] 

# Compute "charged" vertices
if use_hbond:
    vertex_hbond = computeCharges(prot_path.replace(".pdb",""), vertices1, names1)    

# For each surface residue, assign the hydrophobicity of its amino acid. 
if use_hphob:
    vertex_hphobicity = computeHydrophobicity(names1) 

In [ ]:
vertices2 = vertices1
faces2 = faces1
# Fix the mesh.
mesh = pymesh.form_mesh(vertices2, faces2)
mesh = pymesh.submesh(mesh, faces_to_keep, 0)
with io.capture_output() as captured:
    regular_mesh = fix_mesh(mesh, mesh_res)

In [ ]:
regular_mesh, info = pymesh.remove_degenerated_triangles(regular_mesh)

# Compute the normals
vertex_normal = compute_normal(regular_mesh.vertices, regular_mesh.faces, eps=epsilon)

# Assign charges on new vertices based on charges of old vertices (nearest neighbor)
if use_hbond:
    vertex_hbond = assignChargesToNewMesh(regular_mesh.vertices, vertices1, vertex_hbond, feature_interpolation)

if use_hphob:
    vertex_hphobicity = assignChargesToNewMesh(regular_mesh.vertices, vertices1, vertex_hphobicity, feature_interpolation)

if use_apbs:
    vertex_charges = computeAPBS(regular_mesh.vertices, "%s/%s_pocket_%s.pdb"%(workdir, protname, dist_threshold+5), 
                                apbs_bin, pdb2pqr_bin, multivalue_bin, workdir)

In [49]:
workdir = '/home/haotian/Molecule_Generation/Surface-BP/dataset/masif/new_workdir'
vertfile = open("%s/temp1.csv"%workdir, "w")
for vert in regular_mesh.vertices:
    vertfile.write("{},{},{}\n".format(vert[0], vert[1], vert[2]))
vertfile.close()

In [ ]:
vertex_charges.shape

In [ ]:
vertex_charges = computeAPBS(regular_mesh.vertices, "%s/%s_pocket_%s.pdb"%(workdir, protname, dist_threshold+5), 
                            apbs_bin, pdb2pqr_bin, multivalue_bin, workdir)

In [ ]:
import subprocess

In [ ]:
vertices = regular_mesh.vertices
pdb_file = '/tmp/tmpirkhs69l/1z6e_protein_pocket_15.0.pdb'

In [ ]:
cmd = pdb2pqr_bin + " --ff=PARSE --whitespace --noopt --apbs-input %s temp1"%(pdb_file)
p = subprocess.Popen([cmd], shell=True, cwd=workdir)

In [ ]:
cmd = apbs_bin + " temp1.in"
#p = subprocess.Popen([cmd], shell=True, cwd=workdir)

In [ ]:
vertfile = open("%s/temp1.csv"%workdir, "w")
for vert in vertices:
    vertfile.write("{},{},{}\n".format(vert[0], vert[1], vert[2]))
vertfile.close()

In [ ]:
cmd = multivalue_bin + " temp1.csv temp1.dx temp1_out.csv"

In [ ]:
cmd

In [ ]:
workdir